# Exercise 1 (15 marks)

A) Describe a set of 5 potential features for a sequence-labeling classification using CRF. (5
marks)

B) Explain why these features tend to be useful for these types of sequence labeling tasks. (10
marks)

A) Describe a set of 5 potential features for a sequence-labeling classification using CRF. (5
marks)

Five potential features for a sequence-labeling classification using CRF will be:
- The first feature is the Part-of-Speech Tag feature: This feature could provide grammartical information about words in sentences like whether the word is a noun, a verb or an adjective.
- The second feature is word identity feature. This feature represents the identity of individual words in a sequence. For instance, if the word is "swimming" then the word identity feature could be "swim" or even "swimming" itself.
- The third potential feature is the contextual feature. This feature will consider the surrounding words or labels in a sequence like the previous or later words of the target words
- The fourth feature is n-gram feature. This feature represent the local structure of the sequence. For instance, if we know the n-gram (like bi or tri or something else), then that could be an useful feature for sequence-labeling classification.
- The fifth feature is the word shape feature. This feature will be things like the pattern of capitalization or punctuation around the words. This could assist the model in finding the orthographic cues in the data.


B) Explain why these features tend to be useful for these types of sequence labeling tasks. (10 marks)
- The first feature is the Part-of-Speech Tag feature: The reason that this feature is potential is that it could provide grammartical information about words such as their grammartical roles in sentences. By doing so, it could help the model to familiarize with patterns based on words categories so that the model could perform better in tasks like name entity recognition (NER) or chunking.
- The second feature - word identity feature: The reason that make this feature could be useful is because it allows the model to recognize specific words or their base forms, which could be important for identifying sentiments or dequence labels. For instance, in NER tasks, if we know that "Calgary" refers to a location, then it will be beneficial for us to improve the labelling accuracy.
- The third feature - contextual feature: The reason that this feature could be useful is because words are often influenced by their surrounding words as those will likely to give information of the context that the target words is used for. For instance, in chunking task, if we know that a noun is preceded by a determiner, then it's an indicator of the start of a noun phrase.
- The fourth feature is n-gram feature. This feature capture local structure in the documents, which could be useful for understanding the context of the documents that we are examining. For example, in POS tagging, then the bigram "is swimming" is likely to have the POS sequence of VB (verb base form) or VBG which is (Verb Gerund)
- The fifth feature is the words shape feature. This features will encode the visual characteristics of the words such as capitalization, or punctuation around the word. The reason that this feature could be helpful is that it could provide the model the information about the words' orthographic patterns which might assist the model in task like sequence-labeling classification.

# Exercise 2 (20 marks)
You have been provided a basic LSTM for text generation in the notebook provided for this assignment. Use it to answer the questions below.

A) Trace the steps through the LSTM provided and explain in 1-2 sentences what each line is doing (similar to how RNN is commented in the text generation notebook in class). (10
marks)

B) Explain how temperature is expected to impact text generation. Show examples with generated text with temperature set to 0.1, 0.5, 1, and 1.5. Do you see any evidence of temperature impacting the results? (8 marks)

C) Describe 2 benefits a LSTM has over a vanilla RNN. Describe 1 limitation of an LSTM. (2 marks)

**Add brief comments to the LSTM class to explain the purpose of each line of code.**

In [ ]:
import nltk
nltk.download('gutenberg')
import re
from nltk.corpus import gutenberg

# Load text from NLTK Gutenberg corpus
text = gutenberg.raw("carroll-alice.txt")
text = text.replace('\t', ' ').replace('\n', ' ')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [ ]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Define the corpus
# YOU CAN TAKE A SMALL PIECE OF THIS LIST IF YOU EXPERIENCE MEMORY ISSUES
corpus = sent_tokenize(text)

# Tokenize the corpus
tokenizer = lambda x: x.split()
tokenized_corpus = [tokenizer(doc) for doc in corpus]

# Create a vocabulary and dictionary of indices
vocab = list(set([word for doc in tokenized_corpus for word in doc]))
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
vocab_size = len(vocab)

# Convert the corpus to indices
corpus_idx = [[word_to_idx[word] for word in doc] for doc in tokenized_corpus]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Part 2A

In [ ]:
import torch
import torch.nn as nn

# DOCUMENT THE CLASS
# Define the class LSTM which inherits from the nn.Module class from PyTorch, the inclusion of nn.Module will allows
# us to use the functionalities that PyTorch's Module class has.
class LSTM(nn.Module):
    # Initialization of the variables to be used to create LSTM model. This initialization class take 3 parameters:
    # the input_size, the hidden_size and the output_size
    def __init__(self, input_size, hidden_size, output_size):
        # This line call the constructor of the parent class (nn.Module).
        super(LSTM, self).__init__()
        # Store the parameter hidden_size in the class variable also named hidden_size. This will be used later in
        # the creation of the LSTM model
        self.hidden_size = hidden_size
        # Create the LSTM layer. This will be done using the existing lstm layer from PyTorch using nn.LSTM. This
        # layer take 2 inputs which are the input_size and the hidden_state
        self.lstm = nn.LSTM(input_size, hidden_size)
        # This is the creation of the fully connected (linear) layer. It's done using the linear layer from PyTorch
        # it takes 2 inputs which are the hidden_size and the output_size
        self.fc = nn.Linear(hidden_size, output_size)
        # Create a LogSoftmax layer. This layer will apply the LogSoftmax function to the output of the fully
        # connected layer
        self.softmax = nn.LogSoftmax(dim=1)

    # This forward method will defines the forward pass of the LSTM. It take the following as input:
    # the input, the hidden state and the cell state
    def forward(self, input, hidden, cell):
        # Here, we pass the input, hidden state and cell state through the lstm layer. The output that this layer
        # return will be the output, the hidden state and cell state.
        output, (hidden, cell) = self.lstm(input.view(1, 1, -1), (hidden, cell))
        # We pass the output through the fully connected layer. This fully connected layer will return the transformed
        # output
        output = self.fc(output.view(1, -1))
        # Finally, we pass the output through the softmax layer which is applying the LogSoftmax function to the output
        # of the fully connected layer
        output = self.softmax(output)
        # Finally, we return the final output, the new hidden state and the new cell state
        return output, hidden, cell

    # The method to initalize the hidden state and the cell state of the LSTM
    def initHidden(self):
        # It returns 2 tensors contains all zeros values. The size of each tensor will be the same and will be
        # (1,1,hidden_size)
        return torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size)

# We set the batch size to be 32
batch_size=32
# We set the hidden size to be 32
hidden_size=32
# We set the number of training epochs to be 20
num_epoch=20
# We call the class LSTM to create a LSTM model with 3 parameters: the vocab_size, the hidden_size and the vocab_size
model = LSTM(vocab_size, hidden_size, vocab_size)
# We create the optimizer using Adam with the parameters to be the parameter of the newly created LSTM model and the
# learning rate to be 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# I add device to train faster
if (torch.cuda.is_available()):
  device = 'cuda'
else:
  device = 'cpu'

model = model.to(device)

# Train the LSTM model
# Loop over the dataset for num_epoch times
for epoch in range(num_epoch):
  # For each epoch, we divide the data into batches
  for batch_start in range(0, len(corpus_idx), batch_size):
      # We get the current batch of the data
      batch = corpus_idx[batch_start:batch_start + batch_size]
      #  We initialize the hidden and cell state using method initHidden from the class LSTM above
      hidden, cell = model.initHidden()

      # I add this to train faster
      hidden = hidden.to(device)
      cell = cell.to(device)

      # Initialize the loss to be 0 at the beginning of each batch
      loss = 0
      # Create for loop for each element of the batch
      for doc in batch:
          # We loop through each document (but I think it should be range(len(doc)))
          for i in range(len(doc)-1):
              # Initialize the input to be 0 with the size of (1, vocab_size)
              input = torch.zeros(1, vocab_size).to(device)
              # We set the index corresponding to the current word to 1
              input[0, doc[i]] = 1
              # We set the target to be the next word in the document.
              target = torch.tensor([doc[i+1]], dtype=torch.long).to(device)
              # We pass the input, hidden state, and cell state through the model.
              output, hidden, cell = model(input, hidden, cell)
              # We compute the loss between the output and the target.
              loss += nn.functional.nll_loss(output, target)
      # loss /= batch_size because we want the average of the loss, not the cumulative loss
      # For instance, if the loss is [0.8, 0.7], then we want 0.75 as the loss not 1.5
      loss /= batch_size
      # We compute the gradients of the loss with respect to the model parameters.
      loss.backward()
      # We clip the gradients to prevent them from becoming too large.
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
      # Step the optimizer (update the model parameters using the newly computed gradients)
      optimizer.step()
  # Print out the epoch number and its corresponding loss
  print('Epoch {}:, Loss: {:.2f}'.format(epoch+1, loss.item()))

Epoch 1:, Loss: 141.54
Epoch 2:, Loss: 136.26
Epoch 3:, Loss: 132.87
Epoch 4:, Loss: 125.50
Epoch 5:, Loss: 121.73
Epoch 6:, Loss: 115.20
Epoch 7:, Loss: 111.09
Epoch 8:, Loss: 106.26
Epoch 9:, Loss: 99.53
Epoch 10:, Loss: 96.47
Epoch 11:, Loss: 94.12
Epoch 12:, Loss: 90.41
Epoch 13:, Loss: 87.42
Epoch 14:, Loss: 82.68
Epoch 15:, Loss: 77.88
Epoch 16:, Loss: 76.14
Epoch 17:, Loss: 73.73
Epoch 18:, Loss: 71.18
Epoch 19:, Loss: 67.45
Epoch 20:, Loss: 62.78


## Part 2B

In [ ]:
import torch.nn.functional as F

def generate_text(model, start_word, length, temperature=1.0):
    with torch.no_grad():
        # Initialize hidden and cell state
        hidden, cell = model.initHidden()

        # I add this to train faster
        hidden = hidden.to(device)
        cell = cell.to(device)

        # Convert the start word to a tensor
        start_tensor = torch.zeros(1, vocab_size).to(device)
        start_tensor[0, word_to_idx[start_word]] = 1

        # Generate the initial hidden and cell state using the start word
        output, hidden, cell = model(start_tensor, hidden, cell)

        # Sample the next word based on the output probabilities and temperature
        output = output.squeeze().div(temperature).exp().cpu()
        word_idx = torch.multinomial(output, 1).item()

        # Generate the rest of the text
        output_text = [start_word]
        for i in range(length - 1):
            # Convert the previous predicted word to a tensor
            input_tensor = torch.zeros(1, vocab_size).to(device)
            input_tensor[0, word_idx] = 1

            # Generate the next hidden and cell state using the previous predicted word
            output, hidden, cell = model(input_tensor, hidden, cell)

            # Sample the next word based on the output probabilities and temperature
            output = output.squeeze().div(temperature).exp().cpu().to(device)
            word_idx = torch.multinomial(output, 1).item()

            # Convert the predicted word index to a string and add it to the generated text
            output_text.append(idx_to_word[word_idx])

        # End it with a period
        output_text += '.'
        return ' '.join(output_text)

In [ ]:
# Set the seed and generated length
# seed = "As_aliens_entered_our_planet"
seed = "Alice"
text_length = 100

# List of temperature
temperatures = [0.1, 0.5, 1, 1.5]

for temperature in temperatures:
  generated_text = generate_text(model, start_word = seed, length = text_length, temperature=temperature)

  # Print the generated_text
  print(f"Temperature: {temperature}\nGenerated Text: {generated_text}\n\n")

Temperature: 0.1
Generated Text: Alice very much of the reason and said to the Mock Turtle sighed deeply, and began, in a little timidly, 'why you ARE with some time of the Mock Turtle sighed deeply, and began, in a little golden key, and seemed to be down again, and said the Gryphon, and, Alice to Alice very much of the court, 'Bring me the Mock Turtle sighed deeply, and began, in a little golden key, and seemed to be down on her head!' and the Mock Turtle went on. and Alice looked down on the March Hare said to the March Hare said .


Temperature: 0.5
Generated Text: Alice very glad to the Hatter went on. followed her the bottle was no muttering to ask young man said, without opening out again. and she spoke. again. and she looked down on the Queen. and then went on upon her head down and an offended tone, one for a little sister's dream. you'd better not remember like that!' and the Hatter replied. witness at the whole party up like a very melancholy air, and, after folding his tail

* How temperature affect the text generation?

First, let me explain what does temperature determine. So the temperature in the context of NLP will determine the level of randomness in text prediction. It done so by scaling the logits before applying the softmax layer.
The impact of different level of temperature on text generation is:
- With lower temperature, the model will tend to make conservative and deterministic predictions (safe prediction). The model will tend to stick to the training data. However, the output will be less diverse and might be boring.

- With higher temperature value, the model will choose to make predictions in a more random way. As a consequence, the prediction will be more diverse. But the cost we need to pay is that the model might generate incorrect grammar words or incorrect information.

* The impact of temperature on this assignment.

In this assignment, with different temperature value, I saw that the output are different.

With the temperature is 0.1, I see that the generated text is highly repetitive. The sentences here seem almost identical, which is the consequence of us setting the randomness to be low. In addition, we can see the repetition of words like "Mock Turtle".

When we increase the temperature to 0.5, the generated text now becomes more varies compared to when we set the temperature at 0.1. Despite the appearance of repetition words, the sentences are now more coherence and the words here seems more diverse.

At the temperature of 1, I see that the text even be more diverse. The repetition seems not appear in the output at this temperature. However, we now see the drawback of increasing the temperature since some sentences might not make much sense and can be fragmented.

Finally, with the temperature of 1.5, I found the text to be really interesting at first glance, nevertheless, when I actually read the text, I realized that it doesn't make sense. This is the result of the lack of coherence in the generated text. Now, words and phrases are appearing in an unexpected combinations.

## Part 2C

2 benefits of LSTM over vanilla RNN
- The first benefit of LSTM is the ability to capture long-term dependencies. The reason is that the LSTM networks are designed to overcome the vanishing gradient problem. However, vanilla RNN have an issue with capturing long-term dependencies as it can't handle the vanishing or exploding gradient problem. The reason that LSTM can deal with these 2 problem as it has the cell state to maintain information for long periods of time.
- The second benefit is the better performance on sequence data. Since the LSTM model has the ability to capture long-term dependencies, it can have better performance when dealing with sequence data in tasks like machine translation.

1 limitation of LSTM
- The limitation of LSTM compared to vanilla RNN is that it involves more complex computation which lead to longer training time. Since LSTM model involve more complex computations and have more parameters than vanilla RNN, it will take more time to train. This might be a problem when the computing power is limited or the datasets are large.

# Exercise 3 (15 marks)
A) Please submit your generative chatbot made following the in-class activity alongside an example set of dialog between yourself and the chatbot before and after fine-tuning on a character of your choice. It is recommended to use a larger dataset than the one in class for better performance. Describe your results including what happens when you increase the number of epochs to train the model and if there were any limitations with your selected dialogue dataset? (14 marks)

B) What is the maximum sequence length that DialoGPT can accept by default? Hint: the max length is used in the code during conversation generation so you can see how to access it. (1 marks)

In [ ]:
# I put this here so that I could refer to it below that this text has 710 dialogues but my chosen text has less
# I use the script from the movie Interstellar, not Finding Nemo for this assignment

import random
from bs4 import BeautifulSoup
import requests
import re

response = requests.get('https://imsdb.com/scripts/Finding-Nemo.html')
html = response.text

soup = BeautifulSoup(html, "html.parser")
script = soup.find('td', attrs={'class':'scrtext'}).get_text()

# Split the file string into individual dialogues
lines = script.split('\r\n\r\n')

lines = [s.replace('\r\n', ': ') for s in lines]
for line in lines:
  print(line)


------------------------------------------------------------: FINDING NEMO Transcript v1.0: Copyright 2003 Walt Disney Pictures, Pixar Animation Studios: ------------------------------------------------------------: Transcribed by BaD_BURN: email : markgonzalez154@hotmail.com
 ------------------------------------------------------------------: | Okay, this is the work-in-progress FINDING NEMO film transcript.  |: | Why is it 'work-in-progress' you might ask? Well for one, this    |: | isn't a 100% accurate transcript: some words might be missing,    |: | may not be right. Second, some lines may or may not have been     |: | spoken by the right character. There are instances in the film    |: | where a line is spoken but the character isn't on screen, which   |: | makes things complicated. But I'd say this transcript is about    |: | 98-99% accurate. Dialogue for each scene is seperated by a line   |: | of equal signs (=).                                               |: |             

In [ ]:
pattern = r'^[A-Z]+(:\s)?\b'
character = "MARLIN"

marlin_dialog = []
previous_line = ''
for line in lines:
    if line.startswith(character):
      if re.match(pattern, previous_line.strip()) and not previous_line.startswith(character):
        marlin_dialog.append(previous_line)
      marlin_dialog.append(line)
    previous_line = line

marlin_dialog = marlin_dialog[1:]
empty_strings_or_whitespace = [s for s in marlin_dialog if s.strip() == ""]
marlin_dialog = [s for s in marlin_dialog if s.strip()]
marlin_dialog = [s.replace('\b[A-Z]+:', "") for s in marlin_dialog]

print(len(marlin_dialog))

710


In [ ]:
import random
from bs4 import BeautifulSoup
import requests
import re

response = requests.get('https://imsdb.com/scripts/Interstellar.html')
html = response.text

soup = BeautifulSoup(html, "html.parser")
script = soup.find('td', attrs={'class':'scrtext'}).get_text()

# This line is to split the script into individual lines
'''
The reason is that unlike the script of finding Nemo which has "\r\n" to seperate
character and their script, then "\r\n\r\n" between different character's.
The script I am using only contain "\r\n" so if I split "\r\n\r\n" all text are gone
'''
lines = script.split('\n')

# This line is to filter out empty lines
# lines = [line for line in lines if line.strip() != '']
lines = [s.replace('\r\n', ': ') for s in lines]

# This is an empty list to store the formatted lines
formatted_lines = []

# Initialize a variable to hold the current character's name.
'''
So when I have to use "\r\n" to split lines. Then if 1 character have a long script
across many lines. The next line will be treated as a seperate character's dialogue.
So I need to have current character here to make sure that if a character have too long
script. It will still belong to the same character
'''
current_character = None

for line in lines:
    # If the line is the character's name, then we add a colon to the end
    if line.isupper():
        current_character = line.strip() + ":"
    # If not, add the line as dialogue to the current character's line
    else:
        if current_character:
            line = current_character + " " + line.strip()
            current_character = None
        formatted_lines.append(line)

# Finally, we join the lines together with the separator "\n"
filtered_script = '\n'.join(formatted_lines)

print(filtered_script)



                          
                         
                         
INTERSTELLAR: 
                         
                         
                         
                                    Written by
                         
                                  Jonathan Nolan
                         
                         


                         
STORY BY: 
                     Jonathan Nolan, Kip Thorne & Lynda Obst


                         
                         
                         
                         
                         
                         
MARCH 12 2008: 

SPACE.: 
          But not the dark lonely corner of it we're used to. This is
          a glittering inferno -- the center of a distant galaxy.
                         
          Suddenly, something TEARS past at incredible speed: a NEUTRON
          STAR. It SMASHES headlong through everything it encounters...
          planets, stars. Can anything stop this juggernaut?
 

In [ ]:
'''
This block of code is used to combine text of one character into a single line.
The reason is that if 1 character have a long script across many lines.
The next line will be treated as a seperate character's dialogue.
So I need to have current character here to make sure that if a character
have too long script. It will still belong to the same character
'''

# Split the script into lines
lines = filtered_script.split('\n')

# This list will be returned
modified_lines = []

# This is a variable to hold the current line
current_line = ""

for line in lines:
    # If the line contains a colon, we then add the current line to modified_lines and start a new line
    if ":" in line and current_line:
        modified_lines.append(current_line)
        current_line = line
    # Otherwise, we will remove the leading whitespace and append the line to the current line
    else:
        current_line += " " + line.strip()

if current_line:
    modified_lines.append(current_line.strip())

# Join the modified lines back together with '\n' as the separator
# Since the first 3 lines is redundant, we remove it as well
# The first 3 lines is "INTERSTELLAR:     Written by  Jonathan Nolan..."
filtered_script = '\n'.join(modified_lines[3:])

print(filtered_script)

MARCH 12 2008:  
SPACE.:  But not the dark lonely corner of it we're used to. This is a glittering inferno -- the center of a distant galaxy. 
 STAR. It SMASHES headlong through everything it encounters... planets, stars. Can anything stop this juggernaut?  Yes. Something looms at the heart of the galaxy, hidden inside the blinding starlight, a dark flaw in the fabric of
  The neutron star is pulled into the black hole's swirl, spiraling closer and closer to destruction. Finally, it contacts the hole's edge and EXPLODES.  The EXPLOSION is so powerful that it sends shock waves into the fabric of space-time itself. We ride one of these waves, racing back out from the black hole.  Suddenly, a portion of the wave disappears down a crystal- like hole, emerging in a much darker region of the universe -- a backwater that, as the wave races past a giant red planet with a distinctive eye, we recognize as our own.  The wave, now just an infinitesimal ripple, finally reaches our blue planet. It d

In [ ]:
modified_lines[-20]

"(BIG SMILE): We all know about your early life, Mr Cooper. I wrote a paper on it when I was a boy. And I think we found something you'll really enjoy. "

In [ ]:
'''
This block of of code will be used to remove lines like:
"(SMILES): Like every day was Christmas."
'''

lines_without_actions = []

for line in modified_lines:
    # If the line is from a character, add it to lines_without_actions
    if not re.search(r'\(.*\):', line):
        lines_without_actions.append(line)

for line in lines_without_actions:
    print(line)

modified_lines = lines_without_actions

     
INTERSTELLAR:     Written by  Jonathan Nolan     
STORY BY:  Jonathan Nolan, Kip Thorne & Lynda Obst        
MARCH 12 2008:  
SPACE.:  But not the dark lonely corner of it we're used to. This is a glittering inferno -- the center of a distant galaxy. 
 STAR. It SMASHES headlong through everything it encounters... planets, stars. Can anything stop this juggernaut?  Yes. Something looms at the heart of the galaxy, hidden inside the blinding starlight, a dark flaw in the fabric of
  The neutron star is pulled into the black hole's swirl, spiraling closer and closer to destruction. Finally, it contacts the hole's edge and EXPLODES.  The EXPLOSION is so powerful that it sends shock waves into the fabric of space-time itself. We ride one of these waves, racing back out from the black hole.  Suddenly, a portion of the wave disappears down a crystal- like hole, emerging in a much darker region of the universe -- a backwater that, as the wave races past a giant red planet with a distincti

In [ ]:
for modified_line in modified_lines:
  print(modified_line)

     
INTERSTELLAR:     Written by  Jonathan Nolan     
STORY BY:  Jonathan Nolan, Kip Thorne & Lynda Obst        
MARCH 12 2008:  
SPACE.:  But not the dark lonely corner of it we're used to. This is a glittering inferno -- the center of a distant galaxy. 
 STAR. It SMASHES headlong through everything it encounters... planets, stars. Can anything stop this juggernaut?  Yes. Something looms at the heart of the galaxy, hidden inside the blinding starlight, a dark flaw in the fabric of
  The neutron star is pulled into the black hole's swirl, spiraling closer and closer to destruction. Finally, it contacts the hole's edge and EXPLODES.  The EXPLOSION is so powerful that it sends shock waves into the fabric of space-time itself. We ride one of these waves, racing back out from the black hole.  Suddenly, a portion of the wave disappears down a crystal- like hole, emerging in a much darker region of the universe -- a backwater that, as the wave races past a giant red planet with a distincti

In [ ]:
import re

pattern = r'^[A-Z]+(:\s)?\b'
character = "COOPER"

cooper_dialog = []
previous_line = ''
for line in modified_lines:
    if line.startswith(character):
        if previous_line and not previous_line.startswith(character):
            cooper_dialog.append(previous_line.strip())
        cooper_dialog.append(line.strip())
        previous_line = ''
    else:
        previous_line += ' ' + line.strip()

cooper_dialog = [s for s in cooper_dialog if s.strip()]
cooper_dialog = [re.sub(r'^[A-Z]+:', lambda m: m.group().strip(), s) for s in cooper_dialog]
cooper_dialog = cooper_dialog[1:]

print(cooper_dialog)
print(len(cooper_dialog))

['COOPER: Long way to come by bus. I thought you guys would have a plane.', 'BALLPLAYER: We did. Ran out of parts for it. You know anything about diesels?', "COOPER: A little.  Cooper steps to the engine compartment.  The ballplayer notices Cooper's two boys, TOM, 15, and MURPH, 10, watching them. He wanders over. 7.", "BALLPLAYER: You think your dad's going to be able to help us out?  Murph, a filth-encrusted kid with a black eye, smiles at the ballplayer. MURPH: My dad can fix anything. The ballplayer frowns: smartass kid.\r  After a moment under the hood, Cooper signals to the driver, who tries the engine. It turns over once, then STARTS. BALLPLAYER: Sure appreciate the help. the first time: WORLD FAMOUS NEW YORK YANKEES: EXT. SPACE, NEAR EARTH ORBIT:  Earth spins, lazily. From this height, it looks much the same as it has done for thousands of years.  Suddenly, a tiny black object appears, racing toward Earth.  The object SMASHES into a large satellite and races onward. Behind it, 

In [ ]:
import pandas as pd

data = pd.DataFrame(cooper_dialog)
data[['name','line']] = data.iloc[:,0].str.split(':', 1, expand = True)
data

<ipython-input-9-8e9c2b65816b>:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  data[['name','line']] = data.iloc[:,0].str.split(':', 1, expand = True)


,0,name,line
0,COOPER: Long way to come by bus. I thought you...,COOPER,Long way to come by bus. I thought you guys w...
1,BALLPLAYER: We did. Ran out of parts for it. Y...,BALLPLAYER,We did. Ran out of parts for it. You know any...
2,COOPER: A little. Cooper steps to the engine ...,COOPER,A little. Cooper steps to the engine compart...
3,BALLPLAYER: You think your dad's going to be a...,BALLPLAYER,You think your dad's going to be able to help...
4,COOPER: (shakes his head) Satellite. Big one. ...,COOPER,(shakes his head) Satellite. Big one. Probabl...
...,...,...,...
483,COOPER: Curiosity. New level setting. 100 perc...,COOPER,Curiosity. New level setting. 100 percent.
484,ROBOT: Confirmed. Would you like to make any a...,ROBOT,Confirmed. Would you like to make any additio...
485,COOPER: Sense of humor. New level setting. 100...,COOPER,Sense of humor. New level setting. 100 percen...
486,"EXT. FARMHOUSE, COOPER STATION -- TWILIGHT: C...","EXT. FARMHOUSE, COOPER STATION -- TWILIGHT","Cooper sits on his porch, joined by the same..."


In [ ]:
character = "COOPER" #our character of interest
contexted = []

# context window of size 5
n = 5

for i in data[data.name == character].index:
  if i < n:
    continue
  row = []
  prev = i - 1 - n # we additionally substract 1, so row will contain current response and 7 previous responses
  for j in range(i, prev, -1):
    row.append(data.line[j])
  contexted.append(row)

columns = ['response', 'context']
columns = columns + ['context/' + str(i) for i in range(n - 1)]

df = pd.DataFrame.from_records(contexted, columns=columns)
df = df.dropna()
df

,response,context,context/0,context/1,context/2,context/3
0,You ruin it for the boys when you talk like t...,Cooper guides his truck along a potholed roa...,(shakes his head) Satellite. Big one. Probabl...,You think your dad's going to be able to help...,A little. Cooper steps to the engine compart...,We did. Ran out of parts for it. You know any...
1,They didn't have any baseball at all when I w...,I'm not doing my grandkids any favors by lyin...,You ruin it for the boys when you talk like t...,Cooper guides his truck along a potholed roa...,(shakes his head) Satellite. Big one. Probabl...,You think your dad's going to be able to help...
2,I thought okra wasn't susceptible.,: 9. EXT. FARMHOUSE -- MORNING: The sun is ge...,They didn't have any baseball at all when I w...,I'm not doing my grandkids any favors by lyin...,You ruin it for the boys when you talk like t...,Cooper guides his truck along a potholed roa...
3,Something wrong with your truck?,I thought okra wasn't susceptible.,: 9. EXT. FARMHOUSE -- MORNING: The sun is ge...,They didn't have any baseball at all when I w...,I'm not doing my grandkids any favors by lyin...,You ruin it for the boys when you talk like t...
4,What does that have to do with anything?,Something wrong with your truck?,I thought okra wasn't susceptible.,: 9. EXT. FARMHOUSE -- MORNING: The sun is ge...,They didn't have any baseball at all when I w...,I'm not doing my grandkids any favors by lyin...
...,...,...,...,...,...,...
243,No one's heard anything from Brand?,"I don't want you to take this the wrong way, ...",I was hoping to enroll in a course. Try to ge...,"That's admirable, Mr. Cooper. The truth is, m...",Feel like I should be pulling my weight. 150.,Whose plan? ADMINISTRATOR: My assistant tells...
244,Am I really going to hurt anybody by going to...,"I'm sorry. Officially, she's been listed as m...",No one's heard anything from Brand?,"I don't want you to take this the wrong way, ...",I was hoping to enroll in a course. Try to ge...,"That's admirable, Mr. Cooper. The truth is, m..."
245,Curiosity. New level setting. 100 percent.,Listen. Mr. Cooper. You're a hero. You're the...,Am I really going to hurt anybody by going to...,"I'm sorry. Officially, she's been listed as m...",No one's heard anything from Brand?,"I don't want you to take this the wrong way, ..."
246,Sense of humor. New level setting. 100 percen...,Confirmed. Would you like to make any additio...,Curiosity. New level setting. 100 percent.,Listen. Mr. Cooper. You're a hero. You're the...,Am I really going to hurt anybody by going to...,"I'm sorry. Officially, she's been listed as m..."


In [ ]:
!pip -q install transformers==4.22.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 36.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import logging
from transformers.utils import logging as lg
lg.get_logger("transformers").setLevel(logging.ERROR)

model_size = "small" # can change this to 'small' for Colab if needed

tokenizer = AutoTokenizer.from_pretrained(f"microsoft/DialoGPT-{model_size}")
model = AutoModelForCausalLM.from_pretrained(f"microsoft/DialoGPT-{model_size}")

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/351M [00:00<?, ?B/s]

In [ ]:
# I predefined questions so that I don't have to ask DialoGPT manually

questions = [
    "Hey Cooper, what was it like piloting the Endurance spacecraft through the wormhole and exploring those distant planets?",
    "Cooper, how did you feel when you realized the gravity anomaly on Miller's planet caused such a time dilation, resulting in significant time passing on Earth?",
    "What was going through your mind when you encountered Dr. Mann on his frozen planet and discovered his true intentions?",
    "Cooper, can you describe your emotions when you watched the messages from your family after years of separation due to the time dilation effects?",
    "How did you manage to maintain hope and determination throughout your journey, especially when faced with seemingly insurmountable challenges?"
]

# Let's chat with this pretrained model for 5 lines
for step in range(5):
    message = questions[step]

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(message + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(
                bot_input_ids,
                max_length=1000,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=3
            )

    # pretty print last ouput tokens from bot
    print("MESSAGE: {}".format(questions[step]))
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


MESSAGE: Hey Cooper, what was it like piloting the Endurance spacecraft through the wormhole and exploring those distant planets?
DialoGPT: I'm not sure, but I think it was a wormhole.
MESSAGE: Cooper, how did you feel when you realized the gravity anomaly on Miller's planet caused such a time dilation, resulting in significant time passing on Earth?
DialoGPT: It's a worm hole.
MESSAGE: What was going through your mind when you encountered Dr. Mann on his frozen planet and discovered his true intentions?
DialoGPT: What is it?
MESSAGE: Cooper, can you describe your emotions when you watched the messages from your family after years of separation due to the time dilation effects?
DialoGPT: The time diferent
MESSAGE: How did you manage to maintain hope and determination throughout your journey, especially when faced with seemingly insurmountable challenges?
DialoGPT: the time


In [ ]:
import glob, logging, os, pickle, random, re, torch, pandas as pd, numpy as np
from typing import Dict, List, Tuple
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from tqdm.notebook import tqdm, trange
from pathlib import Path
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

logger = logging.getLogger(__name__)

# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = f'output-{model_size}'
        self.model_type = 'gpt2'
        self.model_name_or_path = f'microsoft/DialoGPT-{model_size}'
        self.config_name = f'microsoft/DialoGPT-{model_size}'
        self.tokenizer_name = f'microsoft/DialoGPT-{model_size}'
        self.cache_dir = 'cached'
        self.block_size = 256 # originally 512
        self.per_gpu_train_batch_size = 2 # originally 4
        self.gradient_accumulation_steps = 2 # originally 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 5
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_total_limit = None
        self.seed = 42
        self.local_rank = -1

args = Args()

In [ ]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

def load_and_cache_examples(args, tokenizer, df_trn):
    print("load and cache examples is being run ****************************************")
    return ConversationDataset(tokenizer, args, df_trn)

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)
        directory = args.cache_dir
        cached_features_file = os.path.join(directory, args.model_type + "_cached_lm_" + str(block_size))

        logger.info("Creating features from dataset file at %s", directory)
        self.examples = []
        for _, row in df.iterrows():
            conv = construct_conv(row, tokenizer)
            self.examples.append(conv)

        logger.info("Saving features into cached file %s", cached_features_file)
        with open(cached_features_file, "wb") as handle:
            pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    logger.info("*** Running trainng, Num examples = %d, Num Epochs = %d ***", len(train_dataset), args.num_train_epochs)

    global_step, epochs_trained = 0, 0
    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

    tb_writer.close()

    return global_step, tr_loss / global_step

In [ ]:
def main(df_trn):
    args = Args()

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning("Process rank: %s, device: %s, n_gpu: %s", args.local_rank, device, args.n_gpu)

    set_seed(args) # Set seed

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelForCausalLM.from_pretrained(args.model_name_or_path, from_tf=False, config=config, cache_dir=args.cache_dir)
    model.to(args.device)

    # Training
    train_dataset = load_and_cache_examples(args, tokenizer, df_trn)
    global_step, tr_loss = train(args, train_dataset, model, tokenizer)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    os.makedirs(args.output_dir, exist_ok=True)

    logger.info("Saving model checkpoint to %s", args.output_dir)
    model_to_save = (model.module if hasattr(model, "module") else model)  # Take care of distributed/parallel training
    model_to_save.save_pretrained(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)

    # Good practice: save your training arguments together with the trained model
    torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

    # Load a trained model and vocabulary that you have fine-tuned
    model = AutoModelForCausalLM.from_pretrained(args.output_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
    model.to(args.device)

In [ ]:
for i in range(1000):
  torch.cuda.empty_cache()

In [ ]:
main(df)

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/351M [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

In [ ]:
# Result after training for 5 epochs
# I predefined questions so that I don't have to ask DialoGPT manually

questions = [
    "Hey Cooper, what was it like piloting the Endurance spacecraft through the wormhole and exploring those distant planets?",
    "Cooper, how did you feel when you realized the gravity anomaly on Miller's planet caused such a time dilation, resulting in significant time passing on Earth?",
    "What was going through your mind when you encountered Dr. Mann on his frozen planet and discovered his true intentions?",
    "Cooper, can you describe your emotions when you watched the messages from your family after years of separation due to the time dilation effects?",
    "How did you manage to maintain hope and determination throughout your journey, especially when faced with seemingly insurmountable challenges?"
]

tokenizer = AutoTokenizer.from_pretrained(f'microsoft/DialoGPT-{model_size}')
model = AutoModelForCausalLM.from_pretrained(f'output-{model_size}')

for step in range(5):
    message = questions[step]

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(message + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature = 0.6,
    )

    # pretty print last ouput tokens from bot
    print("MESSAGE: {}".format(questions[step]))
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

MESSAGE: Hey Cooper, what was it like piloting the Endurance spacecraft through the wormhole and exploring those distant planets?
DialoGPT:  You're right. But I'm not going to do that.
MESSAGE: Cooper, how did you feel when you realized the gravity anomaly on Miller's planet caused such a time dilation, resulting in significant time passing on Earth?
DialoGPT:  I didn't know that.  Cooper looks at the map.
MESSAGE: What was going through your mind when you encountered Dr. Mann on his frozen planet and discovered his true intentions?
DialoGPT:  Nothing.  Brand looks at Cooper.
MESSAGE: Cooper, can you describe your emotions when you watched the messages from your family after years of separation due to the time dilation effects?
DialoGPT:  What are you, like a scientist or something?
MESSAGE: How did you manage to maintain hope and determination throughout your journey, especially when faced with seemingly insurmountable challenges?
DialoGPT: !!!?!! Reminds Cooper!!Reminds the man!!...!

In [ ]:
# What happen if I increase epoch to 12
class Args():
    def __init__(self):
        self.output_dir = f'output-{model_size}'
        self.model_type = 'gpt2'
        self.model_name_or_path = f'microsoft/DialoGPT-{model_size}'
        self.config_name = f'microsoft/DialoGPT-{model_size}'
        self.tokenizer_name = f'microsoft/DialoGPT-{model_size}'
        self.cache_dir = 'cached'
        self.block_size = 256 # originally 512
        self.per_gpu_train_batch_size = 2 # originally 4
        self.gradient_accumulation_steps = 2 # originally 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 12
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_total_limit = None
        self.seed = 42
        self.local_rank = -1

args = Args()

In [ ]:
main(df)

load and cache examples is being run ****************************************


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/12 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

Iteration:   0%|          | 0/124 [00:00<?, ?it/s]

In [ ]:
# Retest after increase epoch to 12
# I predefined questions so that I don't have to ask DialoGPT manually

questions = [
    "Hey Cooper, what was it like piloting the Endurance spacecraft through the wormhole and exploring those distant planets?",
    "Cooper, how did you feel when you realized the gravity anomaly on Miller's planet caused such a time dilation, resulting in significant time passing on Earth?",
    "What was going through your mind when you encountered Dr. Mann on his frozen planet and discovered his true intentions?",
    "Cooper, can you describe your emotions when you watched the messages from your family after years of separation due to the time dilation effects?",
    "How did you manage to maintain hope and determination throughout your journey, especially when faced with seemingly insurmountable challenges?"
]

tokenizer = AutoTokenizer.from_pretrained(f'microsoft/DialoGPT-{model_size}')
model = AutoModelForCausalLM.from_pretrained(f'output-{model_size}')

for step in range(5):
    message = questions[step]

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(message + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature = 0.6,
    )

    # pretty print last ouput tokens from bot
    print("MESSAGE: {}".format(questions[step]))
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

MESSAGE: Hey Cooper, what was it like piloting the Endurance spacecraft through the wormhole and exploring those distant planets?
DialoGPT:  I thought I was the only one.  He turns away from her.
MESSAGE: Cooper, how did you feel when you realized the gravity anomaly on Miller's planet caused such a time dilation, resulting in significant time passing on Earth?
DialoGPT:  Nothing.  Cooper looks up from his monitor.
MESSAGE: What was going through your mind when you encountered Dr. Mann on his frozen planet and discovered his true intentions?
DialoGPT:  It's a paradox.
MESSAGE: Cooper, can you describe your emotions when you watched the messages from your family after years of separation due to the time dilation effects?
DialoGPT: !!! Reminds me how I was raised by my great grandkids.!!!!!?!!
MESSAGE: How did you manage to maintain hope and determination throughout your journey, especially when faced with seemingly insurmountable challenges?
DialoGPT: : INT. MURPH'S ROOM, UNDERGROUND FA

*** Describe your results including what happens when you increase the number of epochs to train the model and if there were any limitations with your selected dialogue dataset?

* Describe the result:

From the result of the model before and after fine-tuning following character Cooper, I see that the model's performance in my case have been improved a lot. Before fine-tuning, the model doesn't have any knowledge about things I was asking and it simply put some of the vocabulary I have in the questions to put in its incomplete answer.

However, after fine-tuning for 5 epochs, the model seems to have some base knowledge of the character Joseph Cooper and the movie Interstellar. That's demonstrated in the answers since DialoGPT now mentioned "Cooper" and "journey".

After I increase the number of epoch, the model got a lot more knowledge of the movie Interstellar and the character Joseph Cooper.
This is reflected strongly in the model's answers. Especially, the last answer have shocked me. As when I asked how did Cooper maintain hope along the 20+ years journey, the model accurately stated thing which mean that's the video he send message to Earth for his family with Dr. Brand looks at him. Despite the wording of the model is extremely weird but this have shown that with enough luck, we could somewhat sucessfully train DialoGPT.


* About my selected dataset, there are some limitations.

First of all, I think that the pattern in my chosen dataset is considerably more difficult and complicated compared to the script of the movie "Finding Nemo" demonstrated in the practice notebook of the prof. The reason is that unlike the script of finding Nemo which has "\r\n" to seperate character and their script, then "\r\n\r\n" between different character's. The script I am using only contain "\r\n" so if I split "\r\n\r\n" all text are gone. That's why I need to split then use pattern matching as well as sentence joining to be able to prepare the dataset to fine-tune DialoGPT.

As a consequence, there are sometimes the character's name is extracted wrongly. For instance, the locations like "INT. LAUNCH TUBES, NUCLEAR SUB, UNDERWATER -- DAY" are capitalized the same way as the character. Or with the chosen character Cooper, the text sometimes put his name as "COOPER (CONT'D)" so I need to find a way to remove it so there are more context for the characters for the model to learn from.

Secondly, this text only contain 488 dialogues compared to 710 dialogues in the practice notebook so there will be less information for the model to learn from.

However, I feel that my effort definitely worth it. The reason is that most of the movie script from imsdb.com have the similar structure to the one I am using so by being able to properly extract the text from Interstellar, I will be able to fine-tune DialoGPT following many of the movie from imsdb.com

# Exercise 3B
What is the maximum sequence length that DialoGPT can accept by default?

Answer:
From the hint that the max length is used in the code during conversation generation, we can see that the max sequence length is 1000.
However, when I go to the GitHub repository, I found that the default maximum sequence length for DialoGPT is 128 tokens.
So I think that the default maximum sequence length for DialoGPT is 128 when we loaded the model but during conversation generation of this assignment, it is 1000.
GitHub repo of DialoGPT: https://github.com/microsoft/DialoGPT/blob/master/README.md